In [1]:
from supervised.automl import BaseAutoML, AutoML

In [2]:
from prophet import Prophet

import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

import random
from copy import deepcopy
import math

In [3]:
train_df=pd.read_csv('./data/train.csv')
test_df=pd.read_csv('./data/test.csv')

In [4]:
df=train_df[['date','rental']].copy()
df.columns=['ds','y']
m = Prophet()
m.daily_seasonality=True
m.weekly_seasonality=True
m.yearly_seasonality=True
m.fit(df)
future=m.make_future_dataframe(365)
forecast = m.predict(future)

Initial log joint probability = -73.9282
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       1699.14   5.17586e-06       69.4858      0.7698      0.7698      133   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     110       1699.17   0.000410967       93.1272   3.446e-06       0.001      181  LS failed, Hessian reset 
     150       1699.19   3.83621e-05       77.7678   4.552e-07       0.001      266  LS failed, Hessian reset 
     177       1699.19    1.5188e-07       69.0376      0.2292           1      304   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


In [5]:
train_df['rental']=train_df['rental']/forecast[:-365]['trend']

In [6]:
train_df['date']=pd.to_datetime(train_df['date'])
train_df['year']=train_df['date'].dt.year
train_df['month']=train_df['date'].dt.month
train_df['day']=train_df['date'].dt.day
train_df = train_df[['date', 'precipitation', 'temp_mean', 'temp_highest', 'temp_lowest',
       'PM10', 'PM2.5', 'humidity', 'sunshine_sum', 'sunshine_rate',
       'wind_mean', 'wind_max', 'year', 'month', 'day', 'rental']]

In [7]:
# 옵션에 따라, 선택한 데이터프레임의 컬럼의 결측값을 채워주는 함수
# option = 0, min, max, mean, median
def cal(my_df, column_name, option):
    df=my_df.copy()
    df['date']=pd.to_datetime(df['date'])
    df['month']=df['date'].dt.month
    nn_df = df.iloc[df[column_name].dropna().index]
    month_df_list=[nn_df[nn_df['month']==i] for i in range(1,13)]
    
    if option == 0:
        replace_list=[0 for month_df in month_df_list]
    elif option =='min':
        replace_list=[month_df[column_name].min() for month_df in month_df_list]
    elif option =='max':
        replace_list=[month_df[column_name].max() for month_df in month_df_list]
    elif option == 'mean':
        replace_list=[month_df[column_name].mean() for month_df in month_df_list]
    elif option == 'median':
        replace_list=[month_df[column_name].median() for month_df in month_df_list]
    
    replace_list = ['']+ replace_list
    
    result=[]
    for i in range(my_df.shape[0]):
        n=my_df.iloc[i][column_name]
        if math.isnan(n):
            result.append(replace_list[df.loc[i]['month']])
        else:
            result.append(n)
    my_df[column_name] = result

In [8]:
null_feature = ['precipitation','PM10','PM2.5','sunshine_sum']
best_option_group = ['min' , 'max' , 'median' , 'min']
for column_name, option in zip(null_feature, best_option_group):
    cal(train_df, column_name, option)

In [9]:
random_state_list=[random.randint(0, 100) for _ in range(10)]
random_state_list

[0, 60, 56, 4, 20, 100, 90, 94, 30, 62]

In [10]:
X,y = train_df.iloc[:,1:-1].to_numpy(), train_df.iloc[:,-1].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X,y , test_size=0.2, random_state=123456)

In [11]:
automl = AutoML(mode="Compete", n_jobs=-1, results_path='./hojun8_compete')
automl.fit(X_train, y_train)

AutoML directory: ./hojun8_compete
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['Decision Tree', 'Linear', 'Random Forest', 'Extra Trees', 'LightGBM', 'Xgboost', 'CatBoost', 'Neural Network', 'Nearest Neighbors']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'kmeans_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
1_DecisionTree rmse 0.457712 trained in 0.2 seconds
Adjust validation. Remove: 1_DecisionTree
Validation strategy: 10-fold CV Shuffle
* Step simple_algorithms will try to check up to 4 models
1_DecisionTree rmse 0.405359 trained in 1.0 seconds
2_DecisionTree rmse 0.369951 trained in 0.89 seconds
3_DecisionTree rmse 0.369951 trained in 0.94 seconds
4_L

KeyboardInterrupt: 

In [ ]:
model=AutoML(results_path="./hojun8_compete")

In [ ]:
r2_score(y_test, model.predict(X_test))